In [1]:
import sys
sys.path.append('..')
from ift6758.features import games_to_table
import pandas as pd

In [ ]:
df = pd.DataFrame(games_to_table([2017]))
print(df)

In [ ]:
df.to_csv("../ift6758/data/preprocessed_data/preprocessed_game_data.csv", index=False)

In [ ]:
unique_event_types = df['type'].unique()
unique_event_types

In [ ]:
events_with_no_zone_code = df[df['zoneCode'].isna()]
events_with_no_zone_code

In [ ]:
events_with_no_shot_type = df[df['shotType'].isna()]
events_with_no_shot_type

In [ ]:
events_with_no_goalie = df[df['goalieInNetId'].isna()]
events_with_no_goalie # net empty of no goalie ?

In [ ]:
# errors ?
events_with_no_x_coord = df[df['xCoord'].isna()]
events_with_no_x_coord

In [ ]:
events_with_no_y_coord = df[df['yCoord'].isna()]
events_with_no_y_coord